In [1]:
# for use in tutorial and development; do not include this `sys.path` change in production:
import sys ; sys.path.insert(0, "../")

# Getting Started

First, we'll import the required libraries and add the **PyTextRank** component into the `spaCy` pipeline:

In [2]:
import pytextrank
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank");

Let's take a look at this pipeline now...

In [3]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'textrank']

We can examine the `spaCy` pipeline in much greater detail...

In [4]:
nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'textrank': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  't

Next, let's load some text from a document:

In [5]:
from icecream import ic
import pathlib

text = pathlib.Path("../dat/mih.txt").read_text()
text

'Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types.\n'

Then run the `spaCy` pipeline...

In [6]:
doc = nlp(text)
len(doc)

92

Now we can access the **PyTextRank** component within the `spaCy` pipeline, and use it to get more information for post-processing of the document.
For example, let's see what the elapsed time in milliseconds was for the *TextRank* processing:

In [7]:
tr = doc._.textrank
ic(tr.elapsed_time);

ic| tr.elapsed_time: 7.581949234008789


Let's examine the top-ranked phrases in the document

In [8]:
for phrase in doc._.phrases:
    ic(phrase.rank, phrase.count, phrase.text)
    ic(phrase.chunks)

ic| phrase.rank: 0.18359439311764025
    phrase.count: 1
    phrase.text: 'mixed types'
ic| phrase.chunks: [mixed types]
ic| phrase.rank: 0.1784796193107821
    phrase.count: 3
    phrase.text: 'systems'
ic| phrase.chunks: [systems, systems, systems]
ic| phrase.rank: 0.15037838042245094
    phrase.count: 1
    phrase.text: 'minimal generating sets'
ic| phrase.chunks: [minimal generating sets]
ic| phrase.rank: 0.14740065982407313
    phrase.count: 1
    phrase.text: 'nonstrict inequations'
ic| phrase.chunks: [nonstrict inequations]
ic| phrase.rank: 0.13946027725597837
    phrase.count: 1
    phrase.text: 'strict inequations'
ic| phrase.chunks: [strict inequations]
ic| phrase.rank: 0.1195023546245721
    phrase.count: 1
    phrase.text: 'linear Diophantine equations'
ic| phrase.chunks: [linear Diophantine equations]
ic| phrase.rank: 0.11450088293222845
    phrase.count: 1
    phrase.text: 'natural numbers'
ic| phrase.chunks: [natural numbers]
ic| phrase.rank: 0.10780718173686318
    phra

## Stop Words

To show use of the *stop words* feature, first we'll output a baseline...

In [9]:
text = pathlib.Path("../dat/gen.txt").read_text()
doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='words', chunks=[words, words], count=2, rank=0.16137018222637944)
ic| phrase: Phrase(text='sentences', chunks=[sentences], count=1, rank=0.13367291641220508)
ic| phrase: Phrase(text='Mihalcea et al', chunks=[Mihalcea et al], count=1, rank=0.1095023226326187)
ic| phrase: Phrase(text='et al', chunks=[et al], count=1, rank=0.10745197034799042)
ic| phrase: Phrase(text='Barrios et al', chunks=[Barrios et al], count=1, rank=0.10502825160040344)
ic| phrase: Phrase(text='the remaining words', chunks=[the remaining words], count=1, rank=0.09559863808781449)
ic| phrase: Phrase(text='gensim implements TextRank', chunks=[gensim implements TextRank], count=1, rank=0.09162794519014893)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.08555365347028678)
ic| phrase: Phrase(text='ranking webpages', chunks=[ranking webpages], count=1, rank=0.07894442579092492)
ic| phrase: Phrase(text='algorithm', chunks=[algorithm], count=1, rank=0.07747

Notice how the top-ranked phrase above is `words` ?
Let's add that phrase to our *stop words* list, to exclude it from the ranked phrases...

In [10]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank", config={ "stopwords": { "word": ["NOUN"] } })

doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='sentences', chunks=[sentences], count=1, rank=0.1490464677880926)
ic| phrase: Phrase(text='Mihalcea et al', chunks=[Mihalcea et al], count=1, rank=0.117318519527749)
ic| phrase: Phrase(text='et al', chunks=[et al], count=1, rank=0.11512161354108796)
ic| phrase: Phrase(text='Barrios et al', chunks=[Barrios et al], count=1, rank=0.11252482346188267)
ic| phrase: Phrase(text='gensim implements TextRank', chunks=[gensim implements TextRank], count=1, rank=0.09816426515530181)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.09165889278462461)
ic| phrase: Phrase(text='ranking webpages', chunks=[ranking webpages], count=1, rank=0.08457790386936588)
ic| phrase: Phrase(text='algorithm', chunks=[algorithm], count=1, rank=0.08300479194058319)
ic| phrase: Phrase(text='every other sentence', chunks=[every other sentence], count=1, rank=0.08179233228776425)
ic| phrase: Phrase(text='Okapi BM25 function', chunks=[Okapi BM25 function], 

For each entry, you'll need to add a key that is the *lemma* and a value that's a list of its *part-of-speech* tags.

## Scrubber

Observe how different variations of "sentence", like "every sentence" and "every other sentence", as well as variations of "sentences", occur in phrase list. You can omit such variations by passing a scrubber function in the config.

In [11]:
from spacy.tokens import Span
nlp = spacy.load("en_core_web_sm")


@spacy.registry.misc("prefix_scrubber")
def prefix_scrubber():
	def scrubber_func(span: Span) -> str:
		while len(span) > 1 and span[0].text in ("a", "the", "their", "every", "other"):
			span = span[1:]
		return span.text
	return scrubber_func

nlp.add_pipe("textrank", config={ "stopwords": { "word": ["NOUN"] }, "scrubber": {"@misc": "prefix_scrubber"}})

doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='sentences', chunks=[sentences, the sentences], count=2, rank=0.1490464677880926)
ic| phrase: Phrase(text='Mihalcea et al', chunks=[Mihalcea et al], count=1, rank=0.117318519527749)
ic| phrase: Phrase(text='et al', chunks=[et al], count=1, rank=0.11512161354108796)
ic| phrase: Phrase(text='Barrios et al', chunks=[Barrios et al], count=1, rank=0.11252482346188267)
ic| phrase: Phrase(text='gensim implements TextRank', chunks=[gensim implements TextRank], count=1, rank=0.09816426515530181)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.09165889278462461)
ic| phrase: Phrase(text='ranking webpages', chunks=[ranking webpages], count=1, rank=0.08457790386936588)
ic| phrase: Phrase(text='algorithm', chunks=[algorithm], count=1, rank=0.08300479194058319)
ic| phrase: Phrase(text='sentence', chunks=[every sentence, every other sentence], count=2, rank=0.08179233228776425)
ic| phrase: Phrase(text='Okapi BM25 function', chunks=[Oka

Different variations of "sentence(s)" are now represented as part of single entry in phrase list.

As the scrubber takes in `Spans`, we can also use `toekn.pos_` or any other spaCy `Token` or `Span` attribute in the scrubbing. The variations of "sentences" have different DETs (determiners), so we could achieve a similar result with the folowing scrubber.

In [12]:
@spacy.registry.misc("articles_scrubber")
def articles_scrubber():
    def scrubber_func(span: Span) -> str:
        for token in span:
            if token.pos_ not in ["DET", "PRON"]:
                break
            span = span[1:]
        return span.text
    return scrubber_func

In [13]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank", config={ "stopwords": { "word": ["NOUN"] }, "scrubber": {"@misc": "articles_scrubber"}})

doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='sentences', chunks=[sentences, the sentences], count=2, rank=0.1490464677880926)
ic| phrase: Phrase(text='Mihalcea et al', chunks=[Mihalcea et al], count=1, rank=0.117318519527749)
ic| phrase: Phrase(text='et al', chunks=[et al], count=1, rank=0.11512161354108796)
ic| phrase: Phrase(text='Barrios et al', chunks=[Barrios et al], count=1, rank=0.11252482346188267)
ic| phrase: Phrase(text='gensim implements TextRank', chunks=[gensim implements TextRank], count=1, rank=0.09816426515530181)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.09165889278462461)
ic| phrase: Phrase(text='ranking webpages', chunks=[ranking webpages], count=1, rank=0.08457790386936588)
ic| phrase: Phrase(text='algorithm', chunks=[algorithm], count=1, rank=0.08300479194058319)
ic| phrase: Phrase(text='other sentence', chunks=[every other sentence], count=1, rank=0.08179233228776425)
ic| phrase: Phrase(text='Okapi BM25 function', chunks=[Okapi BM25 fu

We could also use `Span` labels to filter out `ents`, for example, or certain types of entities, e.g. "CARDINAL", or "DATE", if need to do so for our use case.

In [14]:
@spacy.registry.misc("entity_scrubber")
def articles_scrubber():
    def scrubber_func(span: Span) -> str:
        if span[0].ent_type_:
            # ignore named entities
            return "INELIGIBLE_PHRASE"
        return span.text
    return scrubber_func

In [15]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank", config={ "stopwords": { "word": ["NOUN"] }, "scrubber": {"@misc": "entity_scrubber"}})

doc = nlp(text)

for phrase in doc._.phrases[:10]:
    if phrase.text != "INELIGIBLE_PHRASE":
        ic(phrase)

ic| phrase: Phrase(text='sentences', chunks=[sentences], count=1, rank=0.1490464677880926)
ic| phrase: Phrase(text='gensim implements TextRank', chunks=[gensim implements TextRank], count=1, rank=0.09816426515530181)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.09165889278462461)
ic| phrase: Phrase(text='ranking webpages', chunks=[ranking webpages], count=1, rank=0.08457790386936588)
ic| phrase: Phrase(text='algorithm', chunks=[algorithm], count=1, rank=0.08300479194058319)
ic| phrase: Phrase(text='every other sentence', chunks=[every other sentence], count=1, rank=0.08179233228776425)
ic| phrase: Phrase(text='Okapi BM25 function', chunks=[Okapi BM25 function], count=1, rank=0.07919192237459494)
ic| phrase: Phrase(text='original TextRank', chunks=[original TextRank], count=1, rank=0.07346227481329015)
ic| phrase: Phrase(text='TextRank', chunks=[TextRank], count=1, rank=0.07058237377923389)


## GraphViz Export

Let's generate a GraphViz doc `lemma_graph.dot` to visualize the *lemma graph* that **PyTextRank** produced for the most recent document...

In [16]:
tr = doc._.textrank
tr.write_dot(path="lemma_graph.dot")

In [17]:
!ls -lth lemma_graph.dot

-rw-rw-r-- 1 ankushchander ankushchander 17K Jun  4 11:56 lemma_graph.dot


In [18]:
!pip install graphviz

To render this graph, you must first download `GraphViz` <https://www.graphviz.org/download/>

Then you can render a `DOT` file...

In [19]:
import graphviz as gv

gv.Source.from_file("lemma_graph.dot")

Note that the image which gets rendered in a notebook is probably "squished", but other tools can renders these as interesting graphs.

## Altair visualisation

Let's generate an interactive `altair` plot to look at the lemma graph.


In [20]:
!pip install "altair"

In [21]:
tr = doc._.textrank
tr.plot_keyphrases()

alt.Chart(...)

## Extractive Summarization

Again, working with the most recent document above, we'll summarize based on its top `15` phrases, yielding its top `5` sentences...

In [22]:
for sent in tr.summary(limit_phrases=15, limit_sentences=5):
    ic(sent)

ic| sent: First, a quick description of some popular algorithms & implementations for text summarization that exist today: the summarization module in gensim implements TextRank, an unsupervised algorithm based on weighted-graphs from a paper by Mihalcea et al.
ic| sent: Gensim’s TextRank uses Okapi BM25 function to see how similar the sentences are.
ic| sent: It is built on top of the popular PageRank algorithm that Google used for ranking webpages.
ic| sent: Create a graph where vertices are sentences.
ic| sent: Run the PageRank algorithm on the graph.


## Using TopicRank

The *TopicRank* enhanced algorithm is simple to use in the `spaCy` pipeline and it supports the other features described above:

In [23]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("topicrank");

Let's load an example text:

In [24]:
text = pathlib.Path("../dat/cfc.txt").read_text()
text

" Chelsea 'opted against' signing Salomon Rondón on deadline day.\n\nChelsea reportedly opted against signing Salomón Rondón on deadline day despite their long search for a new centre forward. With Olivier Giroud expected to leave, the Blues targeted Edinson Cavani, Dries Mertens and Moussa Dembele – only to end up with none of them. According to Telegraph Sport, Dalian Yifang offered Rondón to Chelsea only for them to prefer keeping Giroud at the club. Manchester United were also linked with the Venezuela international before agreeing a deal for Shanghai Shenhua striker Odion Ighalo. Manager Frank Lampard made no secret of his transfer window frustration, hinting that to secure top four football he ‘needed’ signings. Their draw against Leicester on Saturday means they have won just four of the last 13 Premier League matches."

In [25]:
doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='Salomon Rondón', chunks=[Salomon Rondón, Salomón Rondón, Rondón], count=3, rank=0.07866221348202057)
ic| phrase: Phrase(text='Chelsea', chunks=[Chelsea, Chelsea, Chelsea], count=3, rank=0.06832817272016853)
ic| phrase: Phrase(text='Olivier Giroud', chunks=[Olivier Giroud, Giroud], count=2, rank=0.05574966582168716)
ic| phrase: Phrase(text='deadline day', chunks=[deadline day, deadline day], count=2, rank=0.05008120527495589)
ic| phrase: Phrase(text='Leicester', chunks=[Leicester], count=1, rank=0.039067778208486274)
ic| phrase: Phrase(text='club', chunks=[club], count=1, rank=0.037625206033098234)
ic| phrase: Phrase(text='Edinson Cavani', chunks=[Edinson Cavani], count=1, rank=0.03759951959121995)
ic| phrase: Phrase(text='draw', chunks=[draw], count=1, rank=0.037353607917351345)
ic| phrase: Phrase(text='Manchester United', chunks=[Manchester United], count=1, rank=0.035757812045215435)
ic| phrase: Phrase(text='Dalian Yifang', chunks=[Dalian Yifang], count=1, ra

## Using Biased TextRank

The *Biased TextRank* enhanced algorithm is simple to use in the `spaCy` pipeline and it supports the other features described above:

In [26]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("biasedtextrank");

In [27]:
doc = nlp(text)

focus = "Leicester"
doc._.textrank.change_focus(focus,bias=10.0,  default_bias=0.0)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='Leicester', chunks=[Leicester, Leicester], count=2, rank=0.26184834028994514)
ic| phrase: Phrase(text='Saturday', chunks=[Saturday, Saturday], count=2, rank=0.13938186779355857)
ic| phrase: Phrase(text='the last 13 Premier League matches', chunks=[the last 13 Premier League matches], count=1, rank=0.12502820319236171)
ic| phrase: Phrase(text='none', chunks=[none], count=1, rank=1.9498221604845646e-07)
ic| phrase: Phrase(text='Moussa Dembele', chunks=[Moussa Dembele, Moussa Dembele], count=2, rank=8.640024414329197e-08)
ic| phrase: Phrase(text='Dries Mertens', chunks=[Dries Mertens, Dries Mertens], count=2, rank=5.152284728493906e-08)
ic| phrase: Phrase(text='Edinson Cavani', chunks=[Edinson Cavani], count=1, rank=3.076049036231119e-08)
ic| phrase: Phrase(text='a new centre', chunks=[a new centre], count=1, rank=2.7737546970070932e-08)
ic| phrase: Phrase(text='deadline day', chunks=[deadline day, deadline day], count=2, rank=1.3752326412669907e-08)
ic| phrase: P

The top-ranked phrases from *Biased TextRank* are closely related to the "focus" item: `Leicester`

## Using PositionRank

The *PositionRank* enhanced algorithm is simple to use in the `spaCy` pipeline and it supports the other features described above:

## Using PositionRank

The *PositionRank* enhanced algorithm is simple to use in the `spaCy` pipeline and it supports the other features described above:

In [28]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("positionrank");

In [29]:
doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='deadline day', chunks=[deadline day, deadline day], count=2, rank=0.1671249044190727)
ic| phrase: Phrase(text='Salomon Rondón', chunks=[Salomon Rondón, Salomon Rondón], count=2, rank=0.14836718147498046)
ic| phrase: Phrase(text='Salomón Rondón', chunks=[Salomón Rondón, Salomón Rondón], count=2, rank=0.14169986334846618)
ic| phrase: Phrase(text='Chelsea', chunks=[Chelsea, Chelsea, Chelsea, Chelsea, Chelsea, Chelsea], count=6, rank=0.13419811872859874)
ic| phrase: Phrase(text='Rondón', chunks=[Rondón, Rondón], count=2, rank=0.12722264594603172)
ic| phrase: Phrase(text='a new centre', chunks=[a new centre], count=1, rank=0.09181159181129885)
ic| phrase: Phrase(text='Giroud', chunks=[Giroud, Giroud], count=2, rank=0.0783201596831592)
ic| phrase: Phrase(text='Olivier Giroud', chunks=[Olivier Giroud, Olivier Giroud], count=2, rank=0.07805316118093475)
ic| phrase: Phrase(text='none', chunks=[none], count=1, rank=0.07503538984105931)
ic| phrase: Phrase(text='their long

The top-ranked phrases from *PositionRank* are closely related to the "lead" items: `Chelsea`, `deadline day`, `Salomon Rondón`

## Baseline

Now let's re-run this pipeline with the baseline *TextRank* algorithm to compare results:

In [30]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")
doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='Shanghai Shenhua striker Odion Ighalo', chunks=[Shanghai Shenhua striker Odion Ighalo], count=1, rank=0.11863090071749424)
ic| phrase: Phrase(text='Odion Ighalo', chunks=[Odion Ighalo], count=1, rank=0.10925286108900635)
ic| phrase: Phrase(text='none', chunks=[none], count=1, rank=0.09802416183300769)
ic| phrase: Phrase(text='Moussa Dembele', chunks=[Moussa Dembele, Moussa Dembele], count=2, rank=0.09341044332809736)
ic| phrase: Phrase(text='deadline day', chunks=[deadline day, deadline day], count=2, rank=0.09046182507994752)
ic| phrase: Phrase(text='Dries Mertens', chunks=[Dries Mertens, Dries Mertens], count=2, rank=0.08919649435994934)
ic| phrase: Phrase(text='Edinson Cavani', chunks=[Edinson Cavani], count=1, rank=0.08418633972470349)
ic| phrase: Phrase(text='Shanghai Shenhua', chunks=[Shanghai Shenhua], count=1, rank=0.08254442709505862)
ic| phrase: Phrase(text='Salomon Rondón', chunks=[Salomon Rondón, Salomon Rondón], count=2, rank=0.08228367707127111)
i

The baseline algorithm is picking up named entities, although not emphasizing the order in which these entities were introduced in the text.